In [1]:
%%capture
!pip install sentence-transformers

In [2]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from sentence_transformers import SentenceTransformer, models

## Step 1: use an existing language model
word_embedding_model = models.Transformer('Yunika/muril-base-sentence-transformer')

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer("Yunika/muril-base-sentence-transformer", modules=[word_embedding_model, pooling_model])

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
%%capture
!pip install datasets

In [6]:
from datasets import load_dataset

dataset_id = "stanfordnlp/snli"

dataset = load_dataset(dataset_id)

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [7]:
dataset['validation'].features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [8]:
print(f"- The {dataset_id} dataset has {dataset['train'].num_rows} examples.")
print(f"- Examples look like this: {dataset['train'][0]}")

- The stanfordnlp/snli dataset has 550152 examples.
- Examples look like this: {'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}


In [17]:
from sentence_transformers import InputExample

# Train Dataset
train_examples = []
train_data = dataset['train']
# For agility we only 1/5 of our available data
n_examples = dataset['train'].num_rows // 10
n_examples = 8

for i in range(n_examples):
    example = train_data[i]
    if example['label'] not in [0,1,2]:
        continue
    train_examples.append(InputExample(texts=[example['premise'], example['hypothesis']], label=example['label']))
    

In [18]:
# Evaluation Dataset
evaluation_examples = []
evaluation_data = dataset['validation']
# For agility we only 1/5 of our available data
n_examples = dataset['validation'].num_rows // 4
n_examples = 4

for i in range(n_examples):
    example = train_data[i]
    if example['label'] not in [0,1,2]:
        continue
    evaluation_examples.append(InputExample(texts=[example['premise'], example['hypothesis']], label=example['label']))

In [19]:
print(f"We have a {type(train_examples)} of length {len(train_examples)} containing {type(train_examples[0])}'s.")

We have a <class 'list'> of length 8 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.


In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
evaluator_dataloader = DataLoader(train_examples, shuffle=False, batch_size=2)

In [21]:
from sentence_transformers import losses
from sentence_transformers import evaluation

train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=word_embedding_model.get_word_embedding_dimension(), num_labels=3)
evaluator = evaluation.LabelAccuracyEvaluator(dataloader=evaluator_dataloader, softmax_model=train_loss, )

In [22]:
num_epochs = 5

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [23]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=10
         ) 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
evaluator.csv_file

'accuracy_evaluation_results.csv'

In [26]:
model.save(path="/kaggle/working/")

In [23]:
model.save_to_hub(
    "muril-base-cased-sentence-transformer-snli", 
    organization="sbastola",
    train_datasets=["stanfordnlp/snli"],
    exist_ok=True, 
    )

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

'https://huggingface.co/sbastola/muril-base-cased-sentence-transformer-snli/commit/e52ae906a9837fcce15e4a4d8a2b5be39c9dd22e'